In [14]:
import os 
import sys
while os.path.split(os.getcwd())[1] != 'RecSysChallenge2023-Team':
    os.chdir('..')
sys.path.insert(1, os.getcwd())

################################# IMPORT HyperTuning  #################################
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from skopt.space import Real, Integer, Categorical

################################# IMPORT RECOMMENDERS #################################
from Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender
from Recommenders.IR_feature_weighting import okapi_BM_25

################################## IMPORT LIBRARIES ##################################
import scipy.sparse as sps
from tqdm import tqdm
import pandas as pd
import numpy as np
import similaripy
import math 

import Daniele.Utils.MatrixManipulation as mm
import MyDataManager as dm

from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from sklearn.preprocessing import normalize

URMv = dm.getURMviews()
URMo = dm.getURMopen()
ICMt = dm.getICMt()
ICMl = dm.getICMl()

URM_train_validation, URMv_test = split_train_in_two_percentage_global_sample(URMv, train_percentage = 0.80)

urm =  mm.defaultExplicitURM(URM_train_validation,URMo,add_aug=True,normalize =False)


#Overfittng
urma = mm.augmentURM(mm.explicitURM(URMv))
urm = sps.vstack([URM_train_validation,urma]).tocsr()
urm = similaripy.normalization.bm25plus(urm)
    

icm = mm.augmentedICM(ICMt, ICMl)

recommender = ItemKNN_CFCBF_Hybrid_Recommender(urm,icm)
recommender.fit(topK= 674, shrink= 613, similarity= 'cosine', normalize= True, feature_weighting='TF-IDF')


100%|██████████| 1051828/1051828 [00:00<00:00, 1842518.76it/s]


ItemKNN_CFCBF_HybridRecommender: URM Detected 2 ( 0.0%) users with no interactions.
ItemKNN_CFCBF_HybridRecommender: URM Detected 3461 (12.4%) items with no interactions.
Similarity column 27968 (100.0%), 3262.65 column/sec. Elapsed time 8.57 sec


In [15]:
evaluator_test = EvaluatorHoldout(URMv_test, [10])
ris, _ = evaluator_test.evaluateRecommender(recommender)
ris

EvaluatorHoldout: Ignoring 1526 ( 3.7%) Users that have less than 1 test interactions


/Users/daniele/Desktop/RecSys/RecSysChallenge2023-Team/Evaluation/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),
/Users/daniele/Desktop/RecSys/RecSysChallenge2023-Team/Evaluation/metrics.py:270: RuntimeWarning: invalid value encountered in double_scalars
  ndcg_ = rank_dcg / ideal_dcg


EvaluatorHoldout: Processed 40103 (100.0%) in 25.98 sec. Users per second: 1544


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.088617,0.156318,0.149663,0.054901,0.096314,0.277242,NaN,0.11132,0.448221,0.386311,...,0.963343,0.43179,0.963343,0.177547,11.046966,0.996556,0.493136,0.846025,2.399794,0.432103
